In [55]:
import pandas as pd
import os
import plotly.graph_objects as go
import plotly.express as px
import seaborn as sns

In [56]:
# limpiar datos
ruta_de_archivos = r'C:\Users\rafae\bigData\Dataset de ventas'

archivos = os.listdir(ruta_de_archivos)

dataframes = {}

for archivo in archivos:
    mes = archivo.split('_')[3].split('.')[0]
    ruta_completa = os.path.join(ruta_de_archivos, archivo)
    df = pd.read_csv(ruta_completa)
    df_ventas = df.copy()

    df_ventas['Cantidad Pedida'] = pd.to_numeric(df_ventas['Cantidad Pedida'], errors='coerce')
    df_ventas['Precio Unitario'] = pd.to_numeric(df_ventas['Precio Unitario'], errors='coerce')
    df_ventas['ID de Pedido'] = pd.to_numeric(df_ventas['ID de Pedido'], errors='coerce')


    # eliminar filas
    df_ventas = df_ventas[~df_ventas['Cantidad Pedida'].isna()]
    df_ventas = df_ventas[~df_ventas['Precio Unitario'].isna()]
    df_ventas = df_ventas[~df_ventas['ID de Pedido'].isna()]

    # acomodar columnas
    df_ventas['Cantidad Pedida'] = df_ventas['Cantidad Pedida'].astype(int)
    df_ventas['Precio Unitario'] = df_ventas['Precio Unitario'].astype(int)
    df_ventas['ID de Pedido'] = df_ventas['ID de Pedido'].astype(int)

    dataframes[mes] = df_ventas

meses = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']

dataframes_mes = {}

In [57]:
# Listas para almacenar los totales de ventas y los ingresos
totales_ventas = []
ingresos_totales = []

# Procesar los datos mes a mes
for mes in meses:
    dataframes_mes = dataframes.get(mes)
    
    if dataframes_mes is not None:
        # Sumar ventas totales
        totales_ventas_mes = dataframes_mes['Cantidad Pedida'].sum()
        totales_ventas.append(totales_ventas_mes)

        # Calcular ingresos totales
        dataframes_mes['Ingreso Total'] = dataframes_mes['Cantidad Pedida'] * dataframes_mes['Precio Unitario']
        ingreso_total_mes = dataframes_mes['Ingreso Total'].sum()
        ingresos_totales.append(ingreso_total_mes)

        
    


# Crear figura combinada (barras para ventas, línea para ingresos)
fig = go.Figure()

# Agregar barras para ventas totales
fig.add_trace(go.Bar(
    x=meses,
    y=totales_ventas,
    name='Total de Ventas',
    marker=dict(color='lightblue'),
    text=totales_ventas,
    textposition='auto'
))

# Agregar línea para ingresos totales
fig.add_trace(go.Scatter(
    x=meses,
    y=ingresos_totales,
    name='Ingresos Totales',
    mode='lines+markers',
    line=dict(color='orange', width=3),
    marker=dict(size=8),
    text=[f"${x:.2f}" for x in ingresos_totales],
    textposition='bottom center'
))

# Personalizar el gráfico
fig.update_layout(
    title="Ventas Totales e Ingresos por Mes",
    xaxis_title="Mes",
    yaxis_title="Cantidad",
    template='plotly_dark',
    barmode='group',
    legend=dict(x=0.8, y=1.1),
    plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor='rgba(0,0,0,0)',
    font=dict(color='white'),
)

# Mostrar el gráfico
fig.show()


In [75]:
# Diccionario para almacenar las ventas diarias por mes
ventas_diarias = {}

# Procesar los datos de ventas diarias para cada mes
for mes in meses:
    if mes in dataframes:
        df_mes = dataframes[mes]
        
        # Agrupar por día y sumar la 'Cantidad Pedida'
        ventas_por_dia = df_mes.groupby(df_mes['Fecha de Pedido'].dt.date)['Cantidad Pedida'].sum()
        ventas_diarias[mes] = ventas_por_dia

# Crear gráfico de área apilada para las ventas diarias
fig = go.Figure()

# Agregar áreas apiladas para cada mes
for mes in meses:
    if mes in ventas_diarias:
        fig.add_trace(go.Scatter(
            x=ventas_diarias[mes].index,  # Fechas (días del mes)
            y=ventas_diarias[mes].values,  # Ventas en ese día
            stackgroup='one',  # Agrupar las áreas (apiladas)
            name=mes,  # Nombre del mes en la leyenda
            line=dict(width=0.5),  # Grosor de la línea
            fill='tonexty',  # Rellenar el área
        ))

# Personalizar el gráfico
fig.update_layout(
    title="Ventas Diarias a lo Largo de Cada Mes",
    xaxis_title="Día del Mes",
    yaxis_title="Cantidad Vendida",
    template='plotly_dark',
    plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor='rgba(0,0,0,0)',
    font=dict(color='white'),
    legend=dict(x=0.8, y=1.1),
)


fig.show()


In [59]:
ruta_de_archivos = r'C:\Users\rafae\bigData\Dataset de ventas'

# Listar archivos
archivos = os.listdir(ruta_de_archivos)

dataframes = {}

for archivo in archivos:
    mes = archivo.split('_')[3].split('.')[0]
    ruta_completa = os.path.join(ruta_de_archivos, archivo)
    df = pd.read_csv(ruta_completa)
    df_ventas = df.copy()

    df_ventas['Cantidad Pedida'] = pd.to_numeric(df_ventas['Cantidad Pedida'], errors='coerce')
    df_ventas['Precio Unitario'] = pd.to_numeric(df_ventas['Precio Unitario'], errors='coerce')
    df_ventas['ID de Pedido'] = pd.to_numeric(df_ventas['ID de Pedido'], errors='coerce')

    # eliminar filas vacías
    df_ventas = df_ventas.dropna(subset=['Cantidad Pedida', 'Precio Unitario', 'ID de Pedido'])

    # Acomodar columnas
    df_ventas['Cantidad Pedida'] = df_ventas['Cantidad Pedida'].astype(int)
    df_ventas['Precio Unitario'] = df_ventas['Precio Unitario'].astype(int)
    df_ventas['ID de Pedido'] = df_ventas['ID de Pedido'].astype(int)

    # Convertir 'Fecha de Pedido' a datetime
    df_ventas['Fecha de Pedido'] = pd.to_datetime(df_ventas['Fecha de Pedido'], format='%m/%d/%y %H:%M')

    # Extraer hora
    df_ventas['Hora'] = df_ventas['Fecha de Pedido'].dt.hour

    dataframes[mes] = df_ventas

meses = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio',
         'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']

fig = go.Figure()

# Iterar sobre cada mes y agregar una línea al gráfico
for mes in meses:
    df_mes = dataframes[mes]
    
    # Agrupar por hora
    ventas_por_hora = df_mes.groupby('Hora')['ID de Pedido'].count().sort_index()
    

    fig.add_trace(go.Scatter(
        x=ventas_por_hora.index,
        y=ventas_por_hora.values,
        mode='lines+markers',
        name=mes
    ))

fig.update_layout(
    title='Patrón de Ventas por Hora',
    xaxis_title='Hora del Día',
    yaxis_title='Cantidad de Ventas',
    legend_title='Meses',
    template='plotly_dark'
)

fig.show()

In [60]:
# Unir todos los DataFrames en uno solo para el análisis total
data_completa = pd.concat(dataframes.values())

# Calcular el Ingreso Total
data_completa['Ingreso Total'] = data_completa['Cantidad Pedida'] * data_completa['Precio Unitario']

# Extraer la ciudad y el estado de la dirección de envío
data_completa['Ciudad'] = data_completa['Dirección de Envio'].apply(lambda x: x.split(',')[1].strip())
data_completa['Estado'] = data_completa['Dirección de Envio'].apply(lambda x: x.split(',')[2].split()[0].strip())

# Recorrer cada mes y generar el gráfico de ventas por ciudad
fig_ciudad = go.Figure()

for mes in meses:
    df_mes = dataframes[mes]

    df_mes['Ingreso Total'] = df_mes['Cantidad Pedida'] * df_mes['Precio Unitario']

    # Extraemos la ciudad de la dirección
    df_mes['Ciudad'] = df_mes['Dirección de Envio'].apply(lambda x: x.split(',')[1].strip())

    # Agrupar las ventas por ciudad
    ventas_por_ciudad = df_mes.groupby('Ciudad')['Ingreso Total'].sum().sort_values(ascending=False)

    # Añadir barra al gráfico
    fig_ciudad.add_trace(go.Bar(
        x=ventas_por_ciudad.index[:10],
        y=ventas_por_ciudad.values[:10],
        name=mes
    ))

# Configuración del gráfico de ventas por ciudad
fig_ciudad.update_layout(
    title='Ventas por ciudad',
    xaxis_title='Ciudad',
    yaxis_title='Ingreso Total',
    barmode='group',
    template='plotly_dark'
)

# Mostrar gráfico de ventas por ciudad
fig_ciudad.show()

# Recorrer cada mes y generar el gráfico de ventas por estado
fig_estado = go.Figure()

for mes in meses:
    df_mes = dataframes[mes]

    # Extraer el estado de la dirección
    df_mes['Estado'] = df_mes['Dirección de Envio'].apply(lambda x: x.split(',')[2].split()[0].strip())

    # Agrupar las ventas por estado
    ventas_por_estado = df_mes.groupby('Estado')['Ingreso Total'].sum().sort_values(ascending=False)

    # Añadir barra al gráfico
    fig_estado.add_trace(go.Bar(
        x=ventas_por_estado.index[:10],
        y=ventas_por_estado.values[:10],
        name=mes
    ))

# Configuración del gráfico de ventas por estado
fig_estado.update_layout(
    title='Ventas por estado',
    xaxis_title='Estado',
    yaxis_title='Ingreso Total',
    barmode='group',
    template='plotly_dark'
)

# Mostrar gráfico de ventas por estado
fig_estado.show()

# Generar el mapa coroplético de ventas por estado
ventas_totales_por_estado = data_completa.groupby('Estado')['Ingreso Total'].sum().reset_index()

fig_mapa = px.choropleth(ventas_totales_por_estado, 
                         locations='Estado', 
                         locationmode='USA-states', 
                         color='Ingreso Total',
                         scope="usa",
                         title="Ventas totales por estado en USA")

# Generar el mapa coroplético de ventas por estado con fondo oscuro
fig_mapa = px.choropleth(ventas_totales_por_estado, 
                         locations='Estado', 
                         locationmode='USA-states', 
                         color='Ingreso Total',
                         scope="usa",
                         title="Ventas totales por estado en USA")


# Mostrar mapa coroplético
fig_mapa.show()


In [61]:
# DataFrame vacío para acumular ventas por mes
ventas_productos_global = pd.DataFrame()

# Iteramos sobre los dataframes mensuales
for mes in meses:
    df_mes = dataframes[mes]
    
    # Agrupamos por producto para obtener la cantidad total pedida por cada producto en el mes
    ventas_producto_mes = df_mes.groupby('Producto')['Cantidad Pedida'].sum().reset_index()
    ventas_producto_mes['Mes'] = mes  # Agregamos la columna de mes
    
    # Acumulamos las ventas de productos de cada mes en un DataFrame global
    ventas_productos_global = pd.concat([ventas_productos_global, ventas_producto_mes], ignore_index=True)

# Agrupamos todos los productos por nombre para obtener el total vendido por producto
ventas_totales_productos = ventas_productos_global.groupby('Producto')['Cantidad Pedida'].sum().sort_values(ascending=False)

# Seleccionar los 10 productos más vendidos
top_10_productos = ventas_totales_productos.head(10).index

# Filtrar el DataFrame global para los 10 productos más vendidos
ventas_productos_global_top10 = ventas_productos_global[ventas_productos_global['Producto'].isin(top_10_productos)]

# Crear la figura del gráfico
fig = go.Figure()

# Añadir una barra para cada mes
for mes in meses:
    ventas_mes = ventas_productos_global_top10[ventas_productos_global_top10['Mes'] == mes]
    fig.add_trace(go.Bar(
        x=ventas_mes['Producto'],
        y=ventas_mes['Cantidad Pedida'],
        name=mes
    ))

# Configurar el diseño del gráfico
fig.update_layout(
    title='Top 10 Productos Más Vendidos por Mes',
    xaxis_title='Producto',
    yaxis_title='Cantidad Vendida',
    barmode='stack',
    xaxis_tickangle=-45,
    template='plotly_dark'
)

# Mostrar el gráfico
fig.show()


In [62]:
fig = go.Figure()

# hice mi propia paleta de colores para cada mes
colores_mes = {
    'Enero': 'red',
    'Febrero': 'blue',
    'Marzo': 'green',
    'Abril': 'purple',
    'Mayo': 'orange',
    'Junio': 'pink',
    'Julio': 'brown',
    'Agosto': 'yellow',
    'Septiembre': 'teal',
    'Octubre': 'gray',
    'Noviembre': 'white',
    'Diciembre': 'cyan'
}

for mes in meses:
    df_mes = dataframes[mes]
    
    # Extraemos el día de la semana
    df_mes['Día de la Semana'] = df_mes['Fecha de Pedido'].dt.day_name()
    
    # Agrupar por día de la semana
    ventas_por_dia = df_mes.groupby('Día de la Semana')['Ingreso Total'].sum()
    
    # Asegurar el orden correcto de los días
    dias_ordenados = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    ventas_por_dia = ventas_por_dia.reindex(dias_ordenados)
    
    # trazar las lineas
    fig.add_trace(go.Scatter(
        x=ventas_por_dia.index,
        y=ventas_por_dia.values,
        mode='lines+markers',
        name=mes,
        line=dict(color=colores_mes[mes])  # Asignamos el color específico al mes
    ))

fig.update_layout(
    title='Tendencia de Ventas por Día de la Semana para Todos los Meses',
    xaxis_title='Día de la Semana',
    yaxis_title='Ingresos Totales',
    legend_title='Meses',
    template='plotly_dark'
)

fig.show()

# Crear un gráfico de comparación
fig_dias_laborales = go.Figure()

# Calcular la media de ventas totales para días laborables y fines de semana
dias_laborales = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
ventas_media_dias_laborales = []
ventas_media_fines_semana = []

for mes in meses:
    df_mes = dataframes[mes]
    
    df_mes['Día de la Semana'] = df_mes['Fecha de Pedido'].dt.day_name()
    ventas_por_dia = df_mes.groupby('Día de la Semana')['Ingreso Total'].sum()
    
    # Calcular la media de ventas de días laborables
    total_laborales = ventas_por_dia[dias_laborales].mean()
    # Calcular la media de ventas de fines de semana
    total_fines_semana = ventas_por_dia[['Saturday', 'Sunday']].mean()  # Cambiar suma por media
    
    ventas_media_dias_laborales.append(total_laborales)
    ventas_media_fines_semana.append(total_fines_semana)

# Crear un DataFrame para las comparaciones
df_comparacion = pd.DataFrame({
    'Mes': meses,
    'Media Días Laborables': ventas_media_dias_laborales,
    'Media Fines de Semana': ventas_media_fines_semana
})

# Graficar las diferencias
fig_dias_laborales.add_trace(go.Bar(
    x=df_comparacion['Mes'],
    y=df_comparacion['Media Días Laborables'],
    name='Media Días Laborables',
    marker_color='blue'
))
fig_dias_laborales.add_trace(go.Bar(
    x=df_comparacion['Mes'],
    y=df_comparacion['Media Fines de Semana'],
    name='Media Fines de Semana',
    marker_color='orange'
))

# Configurar el gráfico de días laborables vs fines de semana
fig_dias_laborales.update_layout(
    title='Comparación de la Media de Ventas entre Días Laborables y Fines de Semana',
    xaxis_title='Mes',
    yaxis_title='Ingresos Totales',
    barmode='group',
    legend_title='Tipo de Día',
    template='plotly_dark'
)



In [63]:
# Fechas de eventos especiales (solo un evento por mes)
eventos_especiales = {
    '4 de Julio': '07/04/19',
    'Día de Martin Luther King Jr.': '01/21/19',
    'Día de los Presidentes': '02/18/19',
    'Día de la raza': '10/14/19',
    'Día de los Veteranos': '11/11/19',
    'Halloween': '10/31/19',
    'Super Bowl': '02/03/19',
    'Acción de Gracias': '11/28/19',
    'Navidad': '12/25/19'
}

# Concatenar todos los meses en un DataFrame
df_concatenado = pd.concat([dataframes[mes] for mes in meses], ignore_index=True)

df_concatenado['Fecha de Pedido'] = df_concatenado['Fecha de Pedido'].dt.date

ventas_por_dia = df_concatenado.groupby('Fecha de Pedido')['Ingreso Total'].sum().reset_index()

# Crear un gráfico de líneas
fig = px.line(ventas_por_dia, x='Fecha de Pedido', y='Ingreso Total', title='Ventas Diarias a lo Largo del Año')

# Añadir anotaciones para los eventos especiales
for evento, fecha in eventos_especiales.items():
    fecha_evento = pd.to_datetime(fecha, format='%m/%d/%y').date()  # Convertir la fecha a formato datetime
    ventas_evento = ventas_por_dia[ventas_por_dia['Fecha de Pedido'] == fecha_evento]  # Buscar las ventas en esa fecha
    if not ventas_evento.empty:  # Si existen ventas en esa fecha, añadir anotación
        fig.add_annotation(
            x=fecha_evento,
            y=ventas_evento['Ingreso Total'].values[0],
            text=evento,
            showarrow=True,
            arrowhead=2,
            ax=0,
            ay=-40
        )

fig.update_layout(
    xaxis_title='Fecha',
    yaxis_title='Ingreso Total',
    template='plotly_dark'
)

fig.show()


